In [1]:
from collections import deque, namedtuple
from z3 import Int, unsat, Optimize

In [2]:
with open("../data/2025/day10.txt") as f:
    lines = f.read().splitlines()

In [3]:
Machine = namedtuple('Machine', 'lights_goal joltages_goal buttons')
machines = []
for schema in [[seg for seg in line.split(' ')] for line in lines]:
    machines.append(Machine(
        # Treat the '.#.#' light string as binary
        lights_goal = sum([pow(2, i) for i,v in enumerate(schema[0].strip('[]')) if v == '#']),
        joltages_goal = list(map(int,schema[-1].strip('{}').split(','))),
        buttons = [list(map(int,button.strip('()').split(','))) for button in schema[1:-1]],
    ))

In [4]:
def part1(machine: Machine) -> int:
    queue, visited = deque([(0, 0)]), set()  # Initial state
    while queue: # BFS
        lights, button_presses = queue.popleft()
        for button in machine.buttons:
            new_state = (
                lights ^ sum([pow(2, i) for i in button]), # XOR
                button_presses + 1,
            )
            if new_state in visited: continue
            if new_state[0] == machine.lights_goal: return new_state[1]
            else: visited.add(new_state); queue.append(new_state)

print("Part 1:", sum([part1(machine) for machine in machines])) # 473

Part 1: 473


In [5]:
def part2(machine: Machine) -> int:
    s = Optimize()
    counters = [Int(f'counter_{i}') for i,_ in enumerate(machine.joltages_goal)]
    buttons = [Int(f'button_{i}') for i,_ in enumerate(machine.buttons)]
    # Button presses must not be negative
    for i,_ in enumerate(buttons): s.add(buttons[i] >= 0)
    # Assert joltage counter target as sum of button presses
    for i, joltage in enumerate(machine.joltages_goal):
        sum_of_presses = sum([
            buttons[b] for b, button in enumerate(machine.buttons) if i in button
        ])
        s.add(
            counters[i] == joltage,
            counters[i] == sum_of_presses,
        )
    s.minimize(sum(buttons))
    if unsat == s.check(): return -1
    return sum([s.model()[i].as_long() for i in buttons])

print("Part 2:", sum([part2(machine) for machine in machines])) # 18681

Part 2: 18681
